In [1]:
load("../Imports/GeneralMethods.sage", '../Imports/Utility.sage', '../Imports/Point.sage', '../Imports/Line.sage', '../Imports/TritangentPlane.sage', '../Imports/Group.sage', '../Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(QQ)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f) * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [3]:
# Define cubic SE2, find and classify the lines and compute the 45 tritangent planes

In [4]:
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
SE2 = general_cubic.subs({b:-(c*c+e*f)/c, d:(c*c+e*f)/c})

In [5]:
# Find simmetries

In [6]:
%time adm_SE2 = SE2.find_admissible_projectivities()
len(adm_SE2)

CPU times: user 522 ms, sys: 60.8 ms, total: 583 ms
Wall time: 1.85 s


96

In [7]:
%time simm_SE2 = SE2.find_simmetries(adm_SE2)
len(simm_SE2)

CPU times: user 257 ms, sys: 53.7 ms, total: 310 ms
Wall time: 422 ms


4

In [8]:
# Study how the simmetries permute the Eckardt points

In [9]:
Eck_perms = [Permutation(SE2.apply_proj_to_eck(proj[0])).to_permutation_group_element() for proj in simm_SE2]
Eck_perms_group = PermutationGroup(Eck_perms)
Eck_perms_group.order(), Eck_perms_group.structure_description()

(1, '1')

In [10]:
### geometric interpretation of the group of symmetries

In [11]:
## Action of the stabilizer of SE2 on the 27 lines:
Ga = perm_group_lines(SE2, simm_SE2)

## Action of the stabilizer of SE2 on the 45 tritangent planes:
Gb = perm_group_planes(SE2, simm_SE2)

print("Structure of the group Ga:")
print(PermutationGroup(Ga).structure_description())

print("")

print("Structure of the group Gb:")
print(PermutationGroup(Gb).structure_description())

Structure of the group Ga:
C2 x C2

Structure of the group Gb:
C2 x C2


In [12]:
## Action of the group Ga on the lines: computation of the orbits:
OGa = PermutationGroup(Ga).orbits()

## Action of the group on Gb the planes: computation of the orbits:
OGb = PermutationGroup(Gb).orbits()

In [13]:
# The only fixed line is G4
for orb in OGa:
    if len(orb) == 1:
        print(simbolic_line_from_index(orb[0]))

G4


In [16]:
# The tritangent planes fixed are those through G4
for orb in OGb:
    if len(orb) == 1:
        fixed_plane = simbolic_plane_from_index(orb[0])
        print(fixed_plane, tritangent_plane_from_name(fixed_plane))

t3 ['E1', 'G4', 'F14']
t8 ['E2', 'G4', 'F24']
t13 ['E3', 'G4', 'F34']
t24 ['E5', 'G4', 'F45']
t29 ['E6', 'G4', 'F46']


In [19]:
# The two Eckardt points are fixed
for generator in PermutationGroup(Ga):
    assert image_simbolic_line(generator, 'E1') in ["E1", "F14"]  
    assert image_simbolic_line(generator, 'E2') in ["E2", "F24"]